**CUPED** - метод понижения размерности, основанный на использовании исторических данных и предположения, что текущие данные будут кореллировать с историческими.

Для расчетов надо посчитать новую метрику:

$$
\hat{Y} = Y - \theta (X - \mathbb{E}X)
$$

**Y** - значение метрики во время эксперимента

**X** - ковариата (значение метрики до эксперимента - историческое)

**θ** – коэффициент

$$
\theta = \frac{\text{cov}(Y, X)}{\mathbb{V}X}
$$


**EX** - мат. ожидание (среднее значение метрики)

При подсчете t-теста EX у обоих выборок одинаков, поэтому от него избавляются и  берут:
    
$$ \hat{Y} = Y - \theta X $$

В результате дисперсия новой выборки будет примерно в коэф. корреляция в квадрате раз меньше, чем у исходной.

При этом если тест проводится в течение 4 недель, то ковариату надо брать тоже на 4 неделях до начала теста.

Также нужно рассчитывать **min_sample_size** по формуле MDE, используя cuped-метрику и ее дисперсию.

В этом случае для расчета этой формулы надо взять 4 недели до начала эксп-та для расчета **Y** и 4 недели до этого (8...4 нед до начала эксп-та) для расчета ковариаты.

In [3]:
import numpy as np
import pandas as pd
from scipy import stats
from random import uniform

In [68]:
# генерируем данные:
control = np.random.normal(20, 8, 20000)
test = np.random.normal(50, 7, 20000)

historical_for_test = np.random.uniform(0.7, 0.8, 20000)*test # создаем историч. данные (ковариату) коррелирующую с экспериментом
historical_for_control = np.random.uniform(0.7, 0.8, 20000)*control # создаем историч. данные (ковариату) коррелирующую с экспериментом

control = pd.DataFrame({'historical_metric':historical_for_control, 'experiment_metric':control})
control['group'] = 'control'
test = pd.DataFrame({'historical_metric':historical_for_test, 'experiment_metric':test})
test['group'] = 'test'

data = pd.concat([test, control])
data.sample(4)

,historical_metric,experiment_metric,group
15038,38.680368,51.614627,test
668,33.665167,45.896387,test
1489,12.832447,17.451896,control
11953,16.317417,20.471571,control


In [69]:
# считаем θ:
x = data['historical_metric']
y = data['experiment_metric']
covariance = np.cov(x, y)[0, 1]
variance = x.var()
theta = covariance / variance

theta

1.3224865076658934

In [70]:
# считаем ttest:
metric_cuped_control = data.query('group=="control"')['experiment_metric'] - theta * data.query('group=="control"')['historical_metric']
metric_cuped_test = data.query('group=="test"')['experiment_metric'] - theta * data.query('group=="test"')['historical_metric']
_, pvalue = stats.ttest_ind(metric_cuped_control, metric_cuped_test)

pvalue

2.5568664778085292e-61

In [71]:
# насколько уменьшилась дисперсия:
before = data.query('group=="control"')['experiment_metric'].var()
after = metric_cuped_control.var()

print(f'Дисперсия до: {before:.2f}')
print(f'Дисперсия после CUPED: {after:.2f}')

Дисперсия до: 63.31
Дисперсия после CUPED: 0.66
